## Learning Objective
- Define a train/test split and its importance in evaluating models 

(Cross-validation: evaluating estimator performance)[https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation]

In [92]:
import os
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data. 

This situation is called overfitting. 
To avoid it, it is common practice when performing a (supervised) machine learning experiment to:
1. Build your model only using part of the available data as a train set: `(X_train, y_train)`
1. Use the rest of the available data as a test set: `(X_test, y_test)`. 

In scikit-learn a random split into training and test sets can be quickly computed with the `train_test_split` helper function.

Statsmodels does not have equivalent splitting functionality.  However, we can use a special wrapper class from the internet to use sklearn's function directly. But, we have to l

# This doesn't work with categorical data. Will need to use patsy.dmatrices to get a proper function


# Show students how to run models with categorical data without smf! 

In [26]:
# Original
from sklearn.base import BaseEstimator, RegressorMixin
import statsmodels.api as sm
class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """
    def __init__(self, model_class, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
        
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        self.model_ = self.model_class(y, X)
        self.results_ = self.model_.fit()

    def predict(self, X):
        if self.fit_intercept:
            X = sm.add_constant(X)

        return self.results_.predict(X)
    
X, y = make_regression(random_state=1,  n_features=5, n_samples=100, n_informative=3, noise=20)

print(cross_val_score(SMWrapper(sm.OLS), X, y, scoring='r2')) #output should be identical
print(cross_val_score(LinearRegression(), X, y, scoring='r2')) #output should be identical

[0.92239242 0.93036625 0.72960812 0.91261743 0.90798736]
[0.92239242 0.93036625 0.72960812 0.91261743 0.90798736]


In [72]:
filename = 'insects.csv'

filepath = os.path.join(path, filename)
df = pd.read_csv(filepath, header=1, sep='\s+')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   continent  42 non-null     int64  
 1   latitude   42 non-null     float64
 2   wingsize   42 non-null     int64  
 3   sex        42 non-null     int64  
dtypes: float64(1), int64(3)
memory usage: 1.4 KB


In [73]:
formula = 'wingsize ~ latitude'
y = patsy.dmatrices(formula, df)[0]

In [74]:
X = patsy.dmatrices(formula, df)[1]

In [75]:
print(cross_val_score(SMWrapper(sm.OLS), X, y, scoring='r2', cv=3))
print(cross_val_score(LinearRegression(), X, y, scoring='r2', cv=3))

[-43.46712516  -0.29025087 -26.02432009]
[-43.46712516  -0.29025087 -26.02432009]


In [76]:
formula = 'wingsize ~ latitude + C(sex)'

y, X = patsy.dmatrices(formula, df)

In [77]:
y

DesignMatrix with shape (42, 1)
  wingsize
       901
       896
       906
       907
       898
       893
       913
       915
       927
       924
       930
       905
       889
       915
       930
       895
       926
       944
       925
       920
       934
       797
       806
       812
       807
       818
       809
       810
       819
       800
  [12 rows omitted]
  Terms:
    'wingsize' (column 0)
  (to view full data, use np.asarray(this_obj))

In [78]:
X

DesignMatrix with shape (42, 3)
  Intercept  C(sex)[T.1]  latitude
          1            0      35.5
          1            0      37.0
          1            0      38.6
          1            0      40.7
          1            0      40.9
          1            0      42.4
          1            0      45.0
          1            0      46.8
          1            0      48.8
          1            0      49.8
          1            0      50.8
          1            0      36.4
          1            0      39.3
          1            0      41.3
          1            0      43.4
          1            0      45.5
          1            0      47.3
          1            0      48.5
          1            0      50.4
          1            0      52.1
          1            0      56.1
          1            1      35.5
          1            1      37.0
          1            1      38.6
          1            1      40.7
          1            1      40.9
          1            

In [79]:
print(np.mean(cross_val_score(SMWrapper(sm.OLS), X, y, scoring='r2', cv=10)))
print(np.mean(cross_val_score(LinearRegression(), X, y, scoring='r2', cv=10)))

-0.6267024079010886
-0.6267024079010539


In [61]:
import os
filename = 'cars_multivariate.csv'
path = os.path.join('.','data')

filepath = os.path.join(path, filename)
df = pd.read_csv(filepath)
df['horsepower'] = pd.to_numeric(df['horsepower'], errors='coerce').dropna()

In [67]:
formula = 'mpg ~ horsepower+ weight+ acceleration'
y, X = patsy.dmatrices(formula, df)

In [68]:
y

DesignMatrix with shape (392, 1)
  mpg
   18
   15
   18
   16
   17
   15
   14
   14
   14
   15
   15
   14
   15
   14
   24
   22
   18
   21
   27
   26
   25
   24
   25
   26
   21
   10
   10
   11
    9
   27
  [362 rows omitted]
  Terms:
    'mpg' (column 0)
  (to view full data, use np.asarray(this_obj))

In [69]:
X

DesignMatrix with shape (392, 4)
  Intercept  horsepower  weight  acceleration
          1         130    3504          12.0
          1         165    3693          11.5
          1         150    3436          11.0
          1         150    3433          12.0
          1         140    3449          10.5
          1         198    4341          10.0
          1         220    4354           9.0
          1         215    4312           8.5
          1         225    4425          10.0
          1         190    3850           8.5
          1         170    3563          10.0
          1         160    3609           8.0
          1         150    3761           9.5
          1         225    3086          10.0
          1          95    2372          15.0
          1          95    2833          15.5
          1          97    2774          15.5
          1          85    2587          16.0
          1          88    2130          14.5
          1          46    1835          20.5
 

In [70]:
print(np.mean(cross_val_score(LinearRegression(), X, y, scoring='r2', cv=10)))

0.3915526731610508


In [ ]:
formula = 'wingsize ~ latitude + C(sex)'

In [80]:
import pandas as pd
from sklearn.model_selection import train_test_split
def statmodels_split(df, stratify=None, **kwargs):
    """
    Inputs
    df: pandas dataframe.
        if stratify is None, target column MUST be the first column in the dataframe
        
    stratify: target column or None
    
    Returns: 
    Tuple of dataframes (df_train, df_test) 
    """

    if stratify is None:
        y, X = df.iloc[:,0], df.drop(columns=df.columns[0])
        X_train, X_test, y_train, y_test = train_test_split(X,y, **kwargs)
    else:
        y, X = stratify, df.drop(columns = stratify.name)
        X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, **kwargs)
    
    return pd.concat([X_train, y_train], axis=1), pd.concat([X_test, y_test], axis=1)

In [81]:
filename = 'insects.csv'
path = './data'
filepath = os.path.join(path, filename)
df = pd.read_csv(filepath, header=1, sep='\s+')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   continent  42 non-null     int64  
 1   latitude   42 non-null     float64
 2   wingsize   42 non-null     int64  
 3   sex        42 non-null     int64  
dtypes: float64(1), int64(3)
memory usage: 1.4 KB


In [94]:
# train, test = statmodels_split(df, stratify = df['sex'])
train, test = statmodels_split(df)

In [95]:
formula = 'wingsize ~ latitude + C(sex)'
insect_model = smf.ols(formula=formula, data=train).fit()

In [96]:
insect_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               wingsize   R-squared:                       0.946
Model:                            OLS   Adj. R-squared:                  0.942
Method:                 Least Squares   F-statistic:                     246.4
Date:                Fri, 09 Jul 2021   Prob (F-statistic):           1.68e-18
Time:                        23:54:03   Log-Likelihood:                -121.14
No. Observations:                  31   AIC:                             248.3
Df Residuals:                      28   BIC:                             252.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     824.9323     19.015     43.382      0.000     785.981     863.884
C(sex)[T.1]   -99.5997      4.559    -21.848      0.000    -108.938     -90.262
latitude        1.9859      0.414      4.798      0.000       1.138       2.834
==============================================================================
Omnibus:                        3.568   Durbin-Watson:                   1.376
Prob(Omnibus):                  0.168   Jarque-Bera (JB):                2.194
Skew:                           0.596   Prob(JB):                        0.334
Kurtosis:                       3.529   Cond. No.                         383.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [97]:
y_pred = insect_model.predict(test)
r2_score(test['wingsize'], y_pred)

0.9872426607778263

In [98]:
mean_squared_error(test['wingsize'], y_pred)

32.842873650764005

In [ ]:
references
    * https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation
    * https://stackoverflow.com/questions/41045752/using-statsmodel-estimations-with-scikit-learn-cross-validation-is-it-possible/48949667#48949667